# Hyperparameter Tuning

The flexibility of neural networks is also one of their main drawbacks: there are many hyperparameters to tweak. Not only can you use any imaginable network architecture, but even in a simple MLP you can change the number of layers, the number of neurons per layer, the type of activation function to use in each layer, the weight initialization logic, and much more.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# GridSearchCV or RandomizedSearchCV 

- Simply try as many options as possible and see which one works best on Validation set (or use K-cross validation)

- For this, we need to wrap our Keras models in objects that mimic regular Scikit-Learn regressors.

In [8]:
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)
X_new = X_test[:3]

The options dict is used to ensure that the first layer is properly given the input shape (note that if n_hidden=0, the first layer will be the output layer). It is good practice to provide reasonable defaults to as many hyperparameters as you can, as Scikit-Learn does.

In [3]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    options = { "input_shape": input_shape }
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, **options))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

Next, let’s create a KerasRegressor based on this build_model() function:

In [4]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/home/ravi/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


Now we can use this object like a regular Scikit-Learn regressor: we can train it using its fit() method, then evaluate it using its score() method, and use it to make predictions using its predict() method. Note that any extra parameter you pass to the fit() method will simply get passed to the underlying Keras model. Also note that the score will be the opposite of the MSE because Scikit-Learn wants scores, not losses (i.e., higher should be better).

In [9]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data = (X_valid, y_valid),
              callbacks = [keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 3s 7ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/100
363/363 [==============================] - 2s 6ms/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/100
363/363 [==============================] - 2s 7ms/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/100
363/363 [==============================] - 3s 8ms/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/100
363/363 [==============================] - 3s 7ms/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/100
363/363 [==============================] - 3s 8ms/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/100
363/363 [==============================] - 3s 8ms/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/100
363/363 [==============================] - 3s 8ms/step - loss: 0.4060 - val_l

In [10]:
mse_test = keras_reg.score(X_test, y_test)

162/162 [==============================] - 1s 4ms/step - loss: 0.3342


In [12]:
y_pred = keras_reg.predict(X_new)

In [13]:
y_pred

array([0.6034199, 1.5396053, 4.1040044], dtype=float32)

However, we do not actually want to train and evaluate a single model like this, we want to train hundreds of variants and see which one performs best on the validation set. Since there are many hyperparameters, it is preferable to use a randomized search rather than grid search.

Note that RandomizedSearchCV uses K-fold cross-validation, so it does not use X_valid and y_valid . These are just used for early stopping.

In [15]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden" : [0, 1, 2, 3],
    "n_neurons" : np.arange(1, 100),
    "learning_rate" : reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=3, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
242/242 [==============================] - 2s 8ms/step - loss: 3.9437 - val_loss: 3.9967
Epoch 2/100
242/242 [==============================] - 1s 6ms/step - loss: 2.0232 - val_loss: 4.7805
Epoch 3/100
242/242 [==============================] - 2s 6ms/step - loss: 1.3816 - val_loss: 4.5111
Epoch 4/100
242/242 [==============================] - 2s 6ms/step - loss: 1.1342 - val_loss: 3.5513
Epoch 5/100
242/242 [==============================] - 2s 9ms/step - loss: 0.9782 - val_loss: 2.4922
Epoch 6/100
242/242 [==============================] - 3s 11ms/step - loss: 0.8666 - val_loss: 1.9684
Epoch 7/100
242/242 [==============================] - 2s 9ms/step - loss: 0.7865 - val_loss: 1.5219
Epoch 8/100
242/242 [==============================] - 2s 8ms/step - loss: 0.7309 - val_loss: 1.2091
Epoch 9/100
242/242 [==============================] - 2s 10ms/step - loss: 0.6921 - val_loss: 0.9661
Epoch 10/100
242/242 [==============================] - 2s 8ms/step - loss: 0.6627 - val_

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f5b0324a990>,
                   n_iter=3,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5aa0d1ebd0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

access the best parameters found, the best score, and the trained Keras model like this

In [18]:
rnd_search_cv.best_params_

{'learning_rate': 0.008339092654580042, 'n_hidden': 1, 'n_neurons': 38}

In [19]:
rnd_search_cv.best_score_

-0.3707270125548045

In [20]:
model = rnd_search_cv.best_estimator_.model

In [21]:
model.evaluate(X_test, y_test)

162/162 [==============================] - 1s 4ms/step - loss: 0.3563


0.35634645819664

You can now save this model, evaluate it on the test set, and if you are satisfied with its performance, deploy it to production. Using randomized search is not too hard, and it works well for many fairly simple problems. However, when training is slow (e.g., for more complex problems with larger datasets), this approach will only explore a tiny portion of the hyperparameter space. You can partially alleviate this
problem by assisting the search process manually: first run a quick random search using wide ranges of hyperparameter values, then run another search using smaller ranges of values centered on the best ones found during the first run, and so on. This will hopefully zoom in to a good set of hyperparameters. However, this is very time consuming, and probably not the best use of your time.

Here are a few Python libraries you can use to optimize hyperparameters:
• Hyperopt: a popular Python library for optimizing over all sorts of complex search spaces (including real values such as the learning rate, or discrete values such as the number of layers).
• Hyperas, kopt or Talos: optimizing hyperparameters for Keras model (the first two are based on Hyperopt).
• Scikit-Optimize (skopt): a general-purpose optimization library. The Bayes SearchCV class performs Bayesian optimization using an interface similar to Grid SearchCV .
• Spearmint: a Bayesian optimization library.
• Sklearn-Deap: a hyperparameter optimization library based on evolutionary algorithms, also with a GridSearchCV -like interface.
• And many more!
Moreover, many companies offer services for hyperparameter optimization. For example Google Cloud ML Engine has a hyperparameter tuning service. Other companies provide APIs for hyperparameter optimization, such as Arimo, SigOpt, Oscar and many more.